In [7]:
import pandas as pd
import json

ERRORS = pd.read_csv('/home/mdnikolaev/philurame/Experiments/QuantizationClip/RES/ERRORS.csv', index_col=0)
with open('/home/mdnikolaev/philurame/Experiments/QuantizationClip/RES/CLIP.json', 'r') as f:
  CLIPS = json.load(f)

In [4]:
# TODO (скоро досчитается)

# [TEMP] CLIP calc

In [ ]:
import torch, pickle, sys

sys.path.append('/home/mdnikolaev/philurame/Experiments/QuantizationClip')
from q_utils import *

sys.path.append('/home/mdnikolaev/philurame/SDXL_METRICS')
from main import load_pipe, load_data
from lib.generate_decode import decode_vae
from lib.metric import metrics_registry

coco_data = load_data('/home/mdnikolaev/philurame/SDXL_METRICS/DATA', 'COCO', 100)
pipe      = load_pipe(solver='DDIM', scheduler='LINEAR', cacher_quantizer='NONE', is_optimize=False, add_vae=True)

res = {}

for nbits in [16]:

  # non-quantized metrics
  if nbits == 16:
    latents = torch.load('/home/mdnikolaev/philurame/SDXL_METRICS/DATA/NONE/COCO_50/DDIM_LINEAR.pt')[:100]
    all_layers = get_linear_layers(pipe)
    errors  = [torch.norm(W.weight).item() for W in all_layers]

  # quantized metrics
  else:
    with open(f'/home/mdnikolaev/philurame/Experiments/QuantizationClip/RES/{nbits}.pkl', 'rb') as f:
      errors_latents = pickle.load(f)
      errors  = errors_latents['errors']
      latents = errors_latents['fake_latents']
      
  # decode images
  gen_imgs = decode_vae(pipe, latents)

  # calc CLIP
  clip = metrics_registry['CLIP'](fake_imgs = gen_imgs, real_anns = coco_data.anns)()

  res[nbits] = {
    'errors': errors,
    'clip': clip
  }

import numpy as np
import pandas as pd
import json

df = pd.DataFrame(columns=[])
for nbits in [1,2,3,4,5,6,8]:
  res[nbits]['errors'] = np.array(res[nbits]['errors']) ** 2 / np.array(res[16]['errors']) ** 2
  df[f'errors[{nbits}]'] = res[nbits]['errors']

df.to_csv('/home/mdnikolaev/philurame/Experiments/QuantizationClip/RES/ERRORS.csv')

clips = {i:res[i]['clip'] for i in [1,2,3,4,5,6,8]}
with open('/home/mdnikolaev/philurame/Experiments/QuantizationClip/RES/CLIP.json', 'w') as f:
  json.dump(clips, f)

/tmp/job-2328584/ipykernel_123502/1605200600.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  latents = torch.load('/home/mdnikolaev/philurame/SDXL_METRICS/DATA/NONE/COC